In [3]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

In [2]:
client = QdrantClient(host="localhost", port=6333)

In [6]:
client.create_collection(collection_name="coleccion_climas",
    vectors_config=VectorParams(size=7, distance=Distance.MANHATTAN,
                                datatype='uint8'))

True